# Freeswitch Log RDD

## Altanai Bisht 
( abisht@seattleu.edu
altanai.telecom.com)

License GPL 3

In [1]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

ImportError: PyArrow >= 1.0.0 must be installed; however, it was not found.

In [2]:
from datetime import datetime, date
from pyspark.sql import Row
import re

In [3]:
fileurl="C:\\Users\\abisht\\FS_logs\\*.log"

In [4]:

# This is the regex which is specific to Apache Access Logs parsing  Log formats 
# Example Apache log line:
# 127.0.0.1 -      -    [21/Jul/2014:9:55:27 -0800] "GET /home.html HTTP/1.1" 200 2048 
# 1:IP   2:client 3:user 4:date time              5:method 6:req 7:proto   8:respcode 9:size
APACHE_LOG_PATTERN = '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+) (\S+)" (\d{3}) (\d+)'

#  Freeswitch cdr-csv line:
# "8001","8001","0046423112856","public","2020-06-13 00:58:52","2020-06-13 00:58:52","2020-06-13 00:58:53","1","1","USER_NOT_REGISTERED","f2b05a36-ace2-11ea-aa3b-153da8caf925","","","PCMU","PCMU"

# Freeswitch log line:
# b78e7c94-848b-11ec-986e-1d06bccc4558 2022-02-02 16:53:35.645139 [INFO] mod_dialplan_xml.c:637 Processing remoteclient <altanai>->350000-screen in context public
# a995ab9e-848b-11ec-9864-1d06bccc4558 2022-02-02 16:53:16.325144 [DEBUG] switch_core_state_machine.c:749 (sofia/internal/altanai@10.176.2.43:5061) State DESTROY going to sleep
# FS_LOG_PATTERN = '^(\S+) ((\d{4})-(\d{2})-(\d{2}) ((\d{2}):(\d{2}):(\d{2}))\.\d{6}?) (\[\S+\]) (\S+) (\w+)'
FS_LOG_PATTERN = '^(\S+) ((\d{4})-(\d{2})-(\d{2}) ((\d{2}):(\d{2}):(\d{2}))\.\d{6}?) (\[\S+\]) (\S+) (\w..*)?'

In [5]:
# This functions returns a dictionary containing the parts freeswitch Log.
def parse_log_line(logline):
    logline=  logline.encode('utf-8').decode('ascii', 'ignore')
    print(">>>>>>>")
    print(logline)
    match = re.search(FS_LOG_PATTERN, str(logline))
    if match is None:
        print("Invalid logline: %s" % logline)
        return
    return Row(
        calluuid     = match.group(1),
        date         = match.group(2),
        time         = match.group(3),
        category     = match.group(4),
        module       = match.group(5),
#         msg          = match.group(6),
    )

In [6]:
import sys
import os

def parseLogs():
    """ Read and parse log file """
    parsed_logs = (sc
                   .textFile(fileurl)
                   .map(parse_log_line)
#                    .cache()
                  )

    access_logs = (parsed_logs
                   .filter(lambda s: s[1] == 1)
                   .map(lambda s: s[0])
#                    .cache()
                  )

    failed_logs = (parsed_logs
                   .filter(lambda s: s[1] == 0)
                   .map(lambda s: s[0]))
    failed_logs_count = failed_logs.count()
    if failed_logs_count > 0:
        print('Number of invalid logline: %d' % failed_logs.count())
        for line in failed_logs.take(20):
            print('Invalid logline: %s' % line)

    print('Read %d lines, successfully parsed %d lines, failed to parse %d lines' % (parsed_logs.count(), access_logs.count(), failed_logs.count()))
    return parsed_logs, access_logs, failed_logs


In [7]:
## extra to be removed 
parsed_logs = (sc
               .textFile(fileurl)
               .map(parse_log_line))
parsed_logs.take(200)

NameError: name 'sc' is not defined

In [138]:
import sys
import codecs
print(sys.stdout.encoding)
# if sys.stdout.encoding != 'cp850':
#     sys.stdout = codecs.getwriter('cp850')(sys.stdout.buffer, 'strict')
# if sys.stderr.encoding != 'cp850':
#     sys.stderr = codecs.getwriter('cp850')(sys.stderr.buffer, 'strict')

UTF-8


In [143]:

parsed_logs, access_logs, failed_logs = parseLogs()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 57.0 failed 1 times, most recent failure: Lost task 0.0 in stage 57.0 (TID 84) (20090078-7V3L33.seattleu.edu executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\abisht\Downloads\spark-3.2.1-bin-hadoop3.2\spark-3.2.1-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 619, in main
  File "C:\Users\abisht\Downloads\spark-3.2.1-bin-hadoop3.2\spark-3.2.1-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 609, in process
  File "C:\Users\abisht\Downloads\spark-3.2.1-bin-hadoop3.2\spark-3.2.1-bin-hadoop3.2\python\pyspark\rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "C:\Users\abisht\Downloads\spark-3.2.1-bin-hadoop3.2\spark-3.2.1-bin-hadoop3.2\python\pyspark\rdd.py", line 291

In [113]:
failed_logs = (parsed_logs
               .filter(lambda s: s[1] == 0)
               .map(lambda s: s[0]))
print(failed_logs.count())

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 41.0 failed 1 times, most recent failure: Lost task 0.0 in stage 41.0 (TID 63) (20090078-7V3L33.seattleu.edu executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\abisht\Downloads\spark-3.2.1-bin-hadoop3.2\spark-3.2.1-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 619, in main
  File "C:\Users\abisht\Downloads\spark-3.2.1-bin-hadoop3.2\spark-3.2.1-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 609, in process
  File "C:\Users\abisht\Downloads\spark-3.2.1-bin-hadoop3.2\spark-3.2.1-bin-hadoop3.2\python\pyspark\rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "C:\Users\abisht\Downloads\spark-3.2.1-bin-hadoop3.2\spark-3.2.1-bin-hadoop3.2\python\pyspark\rdd.py", line 291

## RDD creation

PySpark RDD Benefits 
- In-Memory Processing
- Immutability
- Fault Tolerance
- Lazy Evolution
- Partitioning

In [12]:
##  RDD using sparkContext.textFile()
# spark.sparkContext.addFile(url)
# rdd=spark.sparkContext.parallelize(df)

rdd = spark.sparkContext.textFile(fileurl)
# print(rdd.collect())
rdd.take(5)
print(type(rdd))

pyspark.rdd.RDD

In [13]:
print("initial partition count:"+str(rdd.getNumPartitions()))

initial partition count:2


In [63]:
## Repartition and Coalesce -  shuffles data from all nodes 
# reparRdd = rdd.repartition(4)
# print("re-partition count:"+str(reparRdd.getNumPartitions()))

### flatmap

In [23]:
## flatMap – flatMap() transformation flattens the RDD

rdd2 = rdd.flatMap(lambda x: x.split("]")).flatMap(lambda x: x.split("["))
# print(rdd2.collect())
rdd2.take(5)

['2020-07-28 21:28:45.515498 ',
 'NOTICE',
 ' mod_logfile.c:192 New log started: /usr/local/freeswitch/log/freeswitch.log',
 '2022-01-19 20:56:40.842114 ',
 'CONSOLE']

##  map() transformation

In [22]:
rdd3 = rdd2.map(lambda x: (x,1))
rdd3.take(5)

[('2020-07-28 21:28:45.515498 [NOTICE', 1),
 (' mod_logfile.c:192 New log started: /usr/local/freeswitch/log/freeswitch.log',
  1),
 ('2022-01-19 20:56:40.842114 [CONSOLE', 1),
 (' switch_loadable_module.c:1803 Successfully Loaded [mod_logfile', 1),
 ('', 1)]

In [18]:
rdd4 = rdd3.filter(lambda a: a._1.startsWith("INVITE"))

## RDD transformations

In [132]:
df= (cleaned_df
     .groupBy('path')
     .count()
     .orderBy('count',ascending=False))
df.show(5)

NameError: name 'cleaned_df' is not defined

In [11]:
# parsed_rdd = rdd.map(lambda line: parse_log2(line)).filter(lambda line: line[1] == 1).map(lambda line : line[0])

# parsed_rdd2 = parsed_rdd.map(lambda line: map2groups(line))

# rslt=(parsed_rdd2.map(lambda x[5],1))
#                  .reduceByKey(lambda a,b:a+b)
#                  .takeOrdered(10, lambda x: -x[1]))
# rslt

SyntaxError: invalid syntax (Temp/ipykernel_19048/1499920430.py, line 3)

In [ ]:
# import seaborn as sns
# df=pd.DataFrame(rslt,columns=[‘status’,‘count’]) 
# sns.barplot(x=‘status’,y=‘count’,data=df) 
# plt.subplots_adjust(bottom=0.4, right=0.8, top=0.9) 
# plt.xticks(rotation=“vertical”,fontsize=8)

**Refs**
- https://opensource.com/article/19/5/visualize-log-data-apache-spark
- https://sparkbyexamples.com/apache-spark-rdd/spark-rdd-transformations/